# **Visium Mousebrain Example**

**Preprocessing**

In [1]:
import os
import gc
import ot
import anndata
import scanpy as sc
import pandas as pd
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt

import commot as ct

In [2]:
adata = sc.datasets.visium_sge(sample_id='V1_Mouse_Brain_Sagittal_Posterior')

100%|██████████| 9.26M/9.26M [00:00<00:00, 12.7MB/s]
100%|██████████| 20.1M/20.1M [00:01<00:00, 13.4MB/s]


**COMMOT**

In [ ]:
from _utils._pipelinemisc import cellcoms


cellcoms(adata, 'mouse')

In [ ]:
#save to avoid running again
adata.write("mousebrain_COMMOT.h5ad")

**GAM**

In [ ]:
from tools._functions import findgenes, findsignals, masscausalcandidates

interestgenes = findgenes(adata, ngenes = 2000)
interestsignals = findsignals(adata, nsignals=150, filtermincells=10, style='most active')

candidatedf = masscausalcandidates(adata, interestgenes, interestsignals, threshold = 0.0)
candidatedf.head(10)

**Causal Function**

In [ ]:
from tools._functions import masscausal

causalresults = masscausal(adata, candidates=candidatedf, propensity_score_model = 'GAM', model = 'linear', method = 'AIPW', 
           bootstrap=False, teststyle='Naive', n_covars = 20, n_samples = 10000, graph=False)

**Pearson Tiebreaker**

In [ ]:
from tools._functions import pearsonties

pearsonties(adata, causalresults)

**Evaluation**

*Pearsons*

In [ ]:
from _utils._analysis import pearsondf

pearsonresults = pearsondf(adata, interestgenes, interestsignals)

*Dictionary Search*

We need to dictionary search both the causal results and the pearson results.

In [ ]:
from _utils._analysis import dictionarysearch

dictionarysearch(causalresults, pathwaydatabase=mouseRT, receptordatabase=REACTTRUSTmouse_dict)

dictionarysearch(pearsonresults, pathwaydatabase=mouseRT, receptordatabase=REACTTRUSTmouse_dict)

*Power Metrics*

In [ ]:
# make a dataframe to store power metric values
# also calculate n and N (these are the same because they are based on the original data)
powermetrics = pd.DataFrame()
powermetrics['Ns'] = [1*i for i in range(1,500)]
n = findn(adata, pathwaydatabase = mouseRT, receptordatabase=REACTTRUSTmouse_dict)
N = calculateN(adata)
print(n, N)

In [ ]:
# now calculate the power metrics for a variety of selection set sizes
for i in range(len(powermetrics['Ns'])):
        Ns = powermetrics.loc[i, 'Ns']
        small = causalresults.head(Ns)
        pm = powermetric(small, n = n, Ns = Ns, N = N)
        powermetrics.loc[i, "causal results"] = pm

for i in range(len(powermetrics['Ns'])):
        Ns = powermetrics.loc[i, 'Ns']
        small = pearsonresults.head(Ns)
        pm = powermetric(small, n = n, Ns = Ns, N = N)
        powermetrics.loc[i, "pearson results"] = pm

In [ ]:
# plot
plt.figure(figsize = (10, 6))
plt.plot(powermetrics['Ns'], powermetrics['pearson results'], marker='', color='orange', label='Pearsons')
plt.plot(powermetrics['Ns'], powermetrics['causal results'], marker='', color='b', label='Causal')
plt.xlabel('Number of Pairs Selected')
plt.ylabel('Power Metrics')
plt.title('Power Metrics')
plt.legend()
plt.grid(True)
plt.show()

*Positive Matches*

Basically the same thing as above, but looking at correct matches instead of Power Metrics.

In [ ]:
positivematches = pd.DataFrame()
positivematches['Ns'] = [1*i for i in range(1,500)]

for i in range(len(positivematches['Ns'])):
        Ns = positivematches.loc[i, 'Ns']
        small = causalresults.head(Ns)
        matches = calculatens(small)
        positivematches.loc[i, "causal results"] = matches

for i in range(len(positivematches['Ns'])):
        Ns = positivematches.loc[i, 'Ns']
        small = pearsonresults.head(Ns)
        matches = calculatens(small)
        positivematches.loc[i, "pearson results"] = matches

In [ ]:
# plot
plt.figure(figsize = (10, 6))
plt.plot(positivematches['Ns'], positivematches['pearson results'], marker='', color='orange', label='Pearsons')
plt.plot(positivematches['Ns'], positivematches['causal results'], marker='', color='b', label='Causal')
plt.xlabel('Number of Pairs Selected')
plt.ylabel('Positive Matches')
plt.title('Positive Matches')
plt.legend()
plt.grid(True)
plt.show()

*Receptor Filter*

We may also choose to filter out pathways that promote their own receptor. 

In [ ]:
from _utils._analysis import receptorfilter

filteredcausalresults = receptorfilter(causalresults, CellChatmouse_dict)
filteredpearsonresults = receptorfilter(pearsonresults, CellChatmouse_dict)

In [ ]:
# power metrics again
for i in range(len(powermetrics['Ns'])):
        Ns = powermetrics.loc[i, 'Ns']
        small = filteredcausalresults.head(Ns)
        pm = powermetric(small, n = n, Ns = Ns, N = N)
        powermetrics.loc[i, "filtered causal results"] = pm

for i in range(len(powermetrics['Ns'])):
        Ns = powermetrics.loc[i, 'Ns']
        small = filteredpearsonresults.head(Ns)
        pm = powermetric(small, n = n, Ns = Ns, N = N)
        powermetrics.loc[i, "filtered pearson results"] = pm

In [ ]:
# plot
plt.figure(figsize = (10, 6))
plt.plot(powermetrics['Ns'], powermetrics['filtered pearson results'], marker='', color='orange', label='Filtered Pearsons')
plt.plot(powermetrics['Ns'], powermetrics['filtered causal results'], marker='', color='b', label='Filtered Causal')
plt.xlabel('Number of Pairs Selected')
plt.ylabel('Power Metrics')
plt.title('Power Metrics')
plt.legend()
plt.grid(True)
plt.show()

*TF Activity*

We might want to see if the pathway signal pairs identified by our method have the necessary transcription factors present in the dataset. If our method has truly identified causal relationships, then we would expect to see that the pairs have lots of transcription factor present. 

In [ ]:
from _utils._analysis import signalsum, tfactivitysearch

tfactivitysearch(adata, causalresults, pathwaydatabase = mouseRT, receptordatabase = REACTTRUSTmouse_dict, 
                 receptortfdatabase = REACTmouse_dict, tfgenedatabase = TRUSTmouse_dict, pathwayreceptordatabase = CellChatmouse_dict)

tfactivitysearch(adata, pearsonresults, pathwaydatabase = mouseRT, receptordatabase = REACTTRUSTmouse_dict, 
                 receptortfdatabase = REACTmouse_dict, tfgenedatabase = TRUSTmouse_dict, pathwayreceptordatabase = CellChatmouse_dict)

signalsum(causalresults, adata)
signalsum(pearsonresults, adata)

In [ ]:
import matplotlib.pyplot as plt
use = pearsonresults[pearsonresults['dictionary search']=='yes']
plt.scatter(use['Pearson'], use['TF sum']*use['Signal sum'])
plt.xlabel('Pearson')
plt.ylabel('TF*Signal')
plt.title('Pearsons vs. TF*Signal')

In [ ]:
use = causalresults[causalresults['dictionary search']=='yes'].reset_index()
for i in range(0, len(use)):
    use.loc[i, 'negp'] = 1-use.loc[i, 'p_values']
plt.scatter(use['negp'], use['TF sum']*use['Signal sum'])
plt.xlabel('1-p-value')
plt.ylabel('TF*Signal')
plt.title('1-p-value vs. TF*Signal')
plt.legend()

Note that although these metrics are not directly comparable, in both cases it is desirable to have many dots in the upper right corner.